#Step16 복잡한 계산 그래프(구현 편)
##16.1 세대 추가

In [1]:
import numpy as np

class Variable():
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None
    self.generation = 0 # 세대 수를 기록하는 변수 추가

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 # 세대를 기록(부모 세대 + 1)

  def backward(self):
    '''
      여러개의 입출력을 받을 수 있도록 준비.
    '''
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = [self.creator]     
    while funcs :              
      f = funcs.pop()          
      gys = [output.grad for output in f.outputs] # output의 grad 가져옴
      gxs = f.backward(*gys)                      # gxs를 구함
      if not isinstance(gxs, tuple):
        gxs = (gxs, )

      for x, gx in zip(f.inputs, gxs):            # 구한 역전파를 각 x.grad에 저장
        if x.grad is None:                        # 같은 변수일 경우에는 덮어쓰지 말고 더해주자!
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:                 # 계속해서 역전파 수행
          funcs.append(x.creator) 


In [2]:
class Function:
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs) # 언팩
    if not isinstance(ys, tuple): # tuple 형태가 아닐 경우 추가 지원
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]
    self.generation = max([x.generation for x in inputs])

    for output in outputs:
      output.set_creator(self) 
    
    self.inputs = inputs
    self.outputs= outputs 
    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, gy):
    raise NotImplementedError()

def as_array(x):
  if np.isscalar(x):
    # x가 np.float64 같은 scalar 타입인지 확인(일반 float도 확인됨)
    return np.array(x)
  return x

##16.2 세대 순으로 꺼내기

In [4]:
generations = [2, 0, 1, 4, 2]
funcs = []

for g in generations:
  f = Function() # 더미 함수 클래스
  f.generation = g
  funcs.append(f)

[f.generation for f in funcs]

[2, 0, 1, 4, 2]

In [5]:
funcs.sort(key=lambda x: x.generation) # 리스트 정렬
[f.generation for f in funcs]

[0, 1, 2, 2, 4]

In [6]:
f = funcs.pop()
f.generation

4

##16.3 Variable 클래스의 backward

In [7]:
import numpy as np

class Variable():
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None
    self.generation = 0 # 세대 수를 기록하는 변수 추가

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 # 세대를 기록(부모 세대 + 1)

  def backward(self):
    '''
      여러개의 입출력을 받을 수 있도록 준비.
    '''
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:     # 같은 함수가 중복 추가 되는 것을 방지
                                # -> 두 elements가 같은 creator를 가지는 경우!
        funcs.append(f)
        seen_set.add(f)         
        funcs.sort(key=lambda x: x.generation)

    add_func(self.creator)
     
    while funcs :              
      f = funcs.pop()          
      gys = [output.grad for output in f.outputs] # output의 grad 가져옴
      gxs = f.backward(*gys)                      # gxs를 구함
      if not isinstance(gxs, tuple):
        gxs = (gxs, )

      for x, gx in zip(f.inputs, gxs):            # 구한 역전파를 각 x.grad에 저장
        if x.grad is None:                        # 같은 변수일 경우에는 덮어쓰지 말고 더해주자!
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:                 # 계속해서 역전파 수행
          add_func(x.creator)                     # x.creator 추가


##16.4 동작 확인

In [8]:
def as_array(x):
  if np.isscalar(x):
    # x가 np.float64 같은 scalar 타입인지 확인(일반 float도 확인됨)
    return np.array(x)
  return x

class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y

  def backward(self, gy):
    '''
      backward 부분 추가
    '''
    return gy, gy

def add(x0, x1):
  return Add()(x0, x1)

class Square(Function):
  def forward(self, x):
    y = x ** 2
    return y

  def backward(self, gy):
    x = self.inputs[0].data # Function 함수에서 inputs (튜플)로 받기 때문에 input-> inputs[0]으로 바꿔줌
    gx = 2 * x * gy
    return gx

def square(x):
  return Square()(x)

In [9]:
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0
